## Analyza citlivosti
Posunutie hodnot predikcii k 1. nizsej hodnote

In [1]:
#Blok 1
# Nacitanie kniznic a nastavenie seed a zobrazovania riadkov
import numpy as np
import pandas as pd
import seaborn as sns
from keras.models import load_model

np.random.seed(1234)
pd.set_option('display.max_rows', None)

In [6]:
#Blok 2
# Zadefinovanie funkcie pre vytvorenie dat na plotovanie zo samostatnych modelov
def make_prediction_individual(df, name_of_df):
    df = df.copy()

    det_inc = load_model("models/detached_inclination.hdf5")        # inclination
    det_mass_ratio = load_model("models/detached_mass_ratio.hdf5")  # mass ratio
    det_prim_potent = load_model("models/detached_pSP.hdf5")        # primary surface potential
    det_sec_potent = load_model("models/detached_sSP.hdf5")         # secondary surface potential
    det_temp_ratio = load_model("models/detached_t1_t2.hdf5")       # temperature ratio

    # vytvorenie pola kriviek
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    # predikcia
    pred_inc = det_inc.predict(X).flatten()
    pred_q = det_mass_ratio.predict(X).flatten()
    pred_pp = det_prim_potent.predict(X).flatten()
    pred_sp = det_sec_potent.predict(X).flatten()
    pred_tr = det_temp_ratio.predict(X).flatten()

    # vytvorenie vysledneho df predikcii
    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    df.to_csv(f'analyza_citlivosti3/original/{name_of_df}.csv')

In [3]:
# Blok 3
# Nacitanie dat
data = pd.read_pickle("detached_all_parameters.pkl").reset_index()
data.head()

,index,id,curve,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2,filter,critical_surface_potential,primary__equivalent_radius,secondary__equivalent_radius,primary__filling_factor,secondary__filling_factor
0,0,38,"[0.6055271686415179, 0.9842041250556204, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_U,15.09104,0.009996,0.009996,-145.333979,-1502.830354
1,1,38,"[0.608985656265516, 0.9846965713304289, 0.9998...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_B,15.09104,0.009996,0.009996,-145.333979,-1502.830354
2,2,38,"[0.6189025614226916, 0.9837351924934223, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_V,15.09104,0.009996,0.009996,-145.333979,-1502.830354
3,3,38,"[0.6292771409565273, 0.9832675811171884, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_R,15.09104,0.009996,0.009996,-145.333979,-1502.830354
4,4,38,"[0.6543378609145588, 0.9835188424579704, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_I,15.09104,0.009996,0.009996,-145.333979,-1502.830354


In [8]:
# Blok 4
# Najdenie unikatnych hodnot v parametroch inclination, mass ratio, primary potential, secondary potential a pomertu teplot z povodnych dat
inc_set, q_set, omega1_set, omega2_set, t1_t2_set = [], [], [], [], []

for col in data:
    if col == "inclination":
        inc_set = sorted(list(set(data[col])), reverse=True)
    if col == "mass_ratio":
        q_set = sorted(list(set(data[col])), reverse=True)
    if col == "primary__surface_potential":
        omega1_set = sorted(list(set(data[col])), reverse=True)
    if col == "secondary__surface_potential":
        omega2_set = sorted(list(set(data[col])), reverse=True)
    if col == "t1_t2":
        t1_t2_set = sorted(list(set(data[col])), reverse=True)

In [5]:
# Blok 5
# Rozdelenie dat podla filtrov o vzorke 100 kriviek
data_to_predict = data[["id", "curve", "filter", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential", "t1_t2"]]
data_Bessel_U = data_to_predict[data_to_predict["filter"] == "Bessell_U"].sample(100)
data_Bessell_B = data_to_predict[data_to_predict["filter"] == "Bessell_B"].sample(100)
data_Bessell_V = data_to_predict[data_to_predict["filter"] == "Bessell_V"].sample(100)
data_Bessell_R = data_to_predict[data_to_predict["filter"] == "Bessell_R"].sample(100)
data_Bessell_I = data_to_predict[data_to_predict["filter"] == "Bessell_I"].sample(100)
data_SLOAN_u = data_to_predict[data_to_predict["filter"] == "SLOAN_u"].sample(100)
data_SLOAN_g = data_to_predict[data_to_predict["filter"] == "SLOAN_g"].sample(100)
data_SLOAN_r = data_to_predict[data_to_predict["filter"] == "SLOAN_r"].sample(100)
data_SLOAN_i = data_to_predict[data_to_predict["filter"] == "SLOAN_i"].sample(100)
data_SLOAN_z = data_to_predict[data_to_predict["filter"] == "SLOAN_z"].sample(100)
data_Kepler = data_to_predict[data_to_predict["filter"] == "Kepler"].sample(100)
data_GaiaDR2 = data_to_predict[data_to_predict["filter"] == "GaiaDR2"].sample(100)
data_tess = data_to_predict[data_to_predict["filter"] == "TESS"].sample(100)

In [7]:
# Blok 6
# Pouzitie funkcie make_prediction_individual na vytvorenie predikcii pre kazdy filter
make_prediction_individual(data_Bessel_U, "orig_det_data_Bessell_U")
make_prediction_individual(data_Bessell_B, "orig_det_data_Bessell_B")
make_prediction_individual(data_Bessell_V, "orig_det_data_Bessell_V")
make_prediction_individual(data_Bessell_R, "orig_det_data_Bessell_R")
make_prediction_individual(data_Bessell_I, "orig_det_data_Bessell_I")
make_prediction_individual(data_SLOAN_u, "orig_det_data_SLOAN_u")
make_prediction_individual(data_SLOAN_g, "orig_det_data_SLOAN_g")
make_prediction_individual(data_SLOAN_r, "orig_det_data_SLOAN_r")
make_prediction_individual(data_SLOAN_i, "orig_det_data_SLOAN_i")
make_prediction_individual(data_SLOAN_z, "orig_det_data_SLOAN_z")
make_prediction_individual(data_Kepler, "orig_det_data_Kepler")
make_prediction_individual(data_GaiaDR2, "orig_det_data_GaiaDR2")
make_prediction_individual(data_tess, "orig_det_data_TESS")

## Uprava predikcii - posun k najblizsej nizsej hodnote

In [10]:
# Blok 7
# Zadefinovanie funkcie, ktora najde prvu nizsiu hodnotu v zozname unikatnych hodnot pre kazdy parameter
def find_value(number, s):
    for i in range(len(s)-1):
        if number > s[i] and (i != len(s)-1):
            return s[i]
        elif number < s[i] and (i != len(s)-1):
            continue
        else:
            return min(s)

In [12]:
# Blok 8
# funkcia pre vytvorenie dat na plotovanie zo samostatnych modelov a posun hodnot predikcii na 2. vyssiu  hodnotu zo zoznamu unikatnych hodnot
def make_prediction_individual_new(df, name_of_df):
    df = df.copy()

    det_inc = load_model("models/detached_inclination.hdf5")        # inclination
    det_mass_ratio = load_model("models/detached_mass_ratio.hdf5")  # mass ratio
    det_prim_potent = load_model("models/detached_pSP.hdf5")        # primary surface potential
    det_sec_potent = load_model("models/detached_sSP.hdf5")         # secondary surface potential
    det_temp_ratio = load_model("models/detached_t1_t2.hdf5")       # temperature ratio

    # vytvorenie pola kriviek
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    # predikcia
    pred_inc = det_inc.predict(X).flatten()
    pred_q = det_mass_ratio.predict(X).flatten()
    pred_pp = det_prim_potent.predict(X).flatten()
    pred_sp = det_sec_potent.predict(X).flatten()
    pred_tr = det_temp_ratio.predict(X).flatten()

    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    # posun predikcie k nizsej hodnote
    new_preds_inc = []
    for i in df["pred_inc"]:
        new_preds_inc.append(find_value(i, inc_set))
    df['pred_inc'] = new_preds_inc

    new_preds_q = []
    for i in df["pred_q"]:
        new_preds_q.append(find_value(i, q_set))
    df['pred_q'] = new_preds_q

    new_preds_omega1 = []
    for i in df["pred_omega1"]:
        new_preds_omega1.append(find_value(i, omega1_set))
    df['pred_omega1'] = new_preds_omega1

    new_preds_omega2 = []
    for i in df["pred_omega2"]:
        new_preds_omega2.append(find_value(i, omega2_set))
    df['pred_omega2'] = new_preds_omega2

    new_preds_t1_t2 = []
    for i in df["pred_t1_t2"]:
        new_preds_t1_t2.append(find_value(i, t1_t2_set))
    df['pred_t1_t2'] = new_preds_t1_t2

    df.to_csv(f'analyza_citlivosti3/upravene/{name_of_df}.csv')

In [13]:
# Blok 9
# Vytvorenie upravenych predikcii
make_prediction_individual_new(data_Bessel_U, "updated_det_data_Bessell_U")
make_prediction_individual_new(data_Bessell_B, "updated_det_data_Bessell_B")
make_prediction_individual_new(data_Bessell_V, "updated_det_data_Bessell_V")
make_prediction_individual_new(data_Bessell_R, "updated_det_data_Bessell_R")
make_prediction_individual_new(data_Bessell_I, "updated_det_data_Bessell_I")
make_prediction_individual_new(data_SLOAN_u, "updated_det_data_SLOAN_u")
make_prediction_individual_new(data_SLOAN_g, "updated_det_data_SLOAN_g")
make_prediction_individual_new(data_SLOAN_r, "updated_det_data_SLOAN_r")
make_prediction_individual_new(data_SLOAN_i, "updated_det_data_SLOAN_i")
make_prediction_individual_new(data_SLOAN_z, "updated_det_data_SLOAN_z")
make_prediction_individual_new(data_Kepler, "updated_det_data_Kepler")
make_prediction_individual_new(data_GaiaDR2, "updated_det_data_GaiaDR2")
make_prediction_individual_new(data_tess, "updated_det_data_TESS")